In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
def FTBS(c, D, nx, dx, dt, n_total, out_step, BC_method, scheme, save_fig, save_path):
    data = np.zeros((n_total, nx))
    u_past = np.zeros(nx)
    u_now = np.zeros(nx)
    u_next = np.zeros(nx)
    sigma = c * dt / dx
    rho = D * dt / (dx ** 2)
    print(rho, sigma)
    for n in range(n_total):
        if n == 0:
            u_next = np.copy(IC(nx))
            
        else:
            u_next[1:nx-1] = u_now[1:nx-1] + rho*(u_now[2:nx]+u_now[0:nx-2]-2*u_now[1:nx-1]) - sigma*(u_now[1:nx-1]-u_now[0:nx-2])         #FTBS
            BC(BC_method, u_next, u_now)
            
        u_past = np.copy(u_now)
        u_now = np.copy(u_next)
        data[n] = np.copy(u_now)  
        plot_n_save(n, data, path, save_fig, out_step, save_path)
        
def CTCS(c, D, nx, dx, dt, n_total, out_step, BC_method, scheme, save_fig, save_path):
    data = np.zeros((n_total, nx))
    u_past = np.zeros(nx)
    u_now = np.zeros(nx)
    u_next = np.zeros(nx)
    sigma = c * dt / dx
    rho = D * dt / (dx ** 2)
    print(rho, sigma)
    for n in range(n_total):
        if n == 0:
            u_next = np.copy(IC(nx))

        elif n == 1:
        #1st step
            u_next[1:nx-1] = u_now[1:nx-1] + rho*(u_now[2:nx]+u_now[0:nx-2]-2*u_now[1:nx-1]) - sigma*(u_now[1:nx-1]-u_now[0:nx-2])         #FTBS
            BC(BC_method, u_next, u_now) 

        else:
            u_next[1:nx-1] = u_past[1:nx-1] + 2.*rho*(u_now[2:nx]-2*u_now[1:nx-1]+u_now[0:nx-2]) - sigma*(u_now[2:nx]-u_now[0:nx-2])     #CTCS
            BC(BC_method, u_next, u_now)

        u_past = np.copy(u_now)
        u_now = np.copy(u_next)
        data[n] = np.copy(u_now)  
        plot_n_save(n, data, path, save_fig, out_step, save_path)
        np.savetxt('result.txt', data, fmt = '%.9g')

In [3]:
def BC(BC_method, u_next, u_now):
    if BC_method.lower() == 'fixed':          #fixed in time boundary condition
        u_next[0] = u_now[0]                  #BC@x=0
        u_next[-1] = u_now[-1]                #BC@x=L
        
    if BC_method.lower() == 'periodic':       #periodic boundary condition
        u_next[0] = u_next[-2]                #u_0=u_L-1
        u_next[-1] = u_next[1]                #u_L=u_1

In [4]:
def Advection_Diffusion(c, D, nx, dx, dt, n_total, out_step, BC_method, scheme, save_fig, save_path):
    if scheme == 'FTBS':
        FTBS(c, D, nx, dx, dt, n_total, out_step, BC_method, scheme, save_fig, save_path)
    if scheme == 'CTCS':
        CTCS(c, D, nx, dx, dt, n_total, out_step, BC_method, scheme, save_fig, save_path)    

In [25]:
def plot_n_save(n, data, path, save, out_step, save_fig):
    if n%out_step == 0:
        title = str(n).zfill(5)
        filename = path + title + f'.png'
        fig = plt.figure(figsize = (6, 4), dpi = 80)
        ax = fig.add_subplot()
        ax.set_xlim(0,600)
        ax.set_ylim(-0.5,1.5)
        ax.set_ylabel('u (mag)', fontsize = 15)
        ax.set_xlabel('x (km)', fontsize = 15)
        ax.set_title(title, fontsize = 15)
        ax.set_facecolor('#E6E6E6')
        matrix=[data[n] for i in range(50)]
        ax=sns.heatmap(data=matrix, vmin=0, vmax=1)
        # sns.lineplot(data = data[n])
        # sns.lineplot(data = data[0])
        if save:
            plt.savefig(filename, dpi = 300)
        else:
            plt.show()
        plt.close()

In [28]:
def make_movie(movie_name, fig_folder):
    images = [img for img in sorted(os.listdir(fig_folder)) if img.endswith(".png")]
    frame = cv2.imread(os.path.join(fig_folder, images[0]))
    height, width, layers = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(movie_name, fourcc, 20, (width, height))

    for image in images:
        video.write(cv2.imread(os.path.join(fig_folder, image)))

    cv2.destroyAllWindows()
    video.release()


In [7]:
def IC(nx):
    IC = np.zeros(nx)                 #IC initialize
    
    # 1. A box-shaped packet located at the center
    IC[nx//2-10:nx//2+10] = 1.        #IC
    
    # 2. A bell-shaped packet located at the center
    # x = np.linspace(0, nx, num = int(nx))
    # IC = np.exp(-8e-4*(x-nx/2)**2)
    
    # 3. A sinusodal shape packet
    # IC[nx//2-10:nx//2+10] = np.cos(np.linspace(-1., 1., 20))   
    return IC

In [26]:
nx = 601                # number of grid points
dx = 1000.0             # grid spacing = 1km
dt = 30.0               # time interval = 30 second
n_total = 9000          # final time step
out_step = 100          # number of time steps between output
                        # output is written every 30s x 50 = 25 mins
D = 2000.               # diffustion constant
c = 15.                 # propagation speed > 0 (upwind) 
path = os.getcwd() + '\\figs\\'
BC_method = 'periodic'
scheme = 'FTBS'
save_fig = True
# rho, sigma              # rho, sigma <= 1 for stability

Advection_Diffusion(c, D, nx, dx, dt, n_total, out_step, BC_method, scheme, save_fig, path)

0.06 0.45


In [27]:
movie_name = 'movie.mp4v'
fig_folder = 'figs'
make_movie(movie_name, fig_folder)